### Практическая работа 2

### Часть 1. Задание

Программная реализация симметричного блочного шифра "Кузнечик" (ГОСТ Р 34.12-2015). Программа должна:

1) принимать на вход файл, содержащий открытый текст, подлежащий зашифрованию, или шифртекст, подлежащий расшифрованию;
2) принимать на вход секретный ключ;
3) [дополнительная опция, не являющаяся обязательной] давать пользователю возможность выбирать режим работы блочного шифра;
4) осуществлять зашифрование или расшифрование выбранного файла по выбору пользователя и сохранять результат в новом файле.

### Часть 2. Краткая теоретическая часть;

«Кузнечик» (англ. Kuznyechik[1] или англ. Kuznechik[2][3]) — симметричный алгоритм блочного шифрования с размером блока 128 бит и длиной ключа 256 бит, использующий для генерации раундовых ключей SP-сеть.

**Описание алгоритма**. Для шифрования, расшифрования и генерации ключа используются следующие функции:
1) XOR с раундовым ключом
2) Нелинейное биективное преобразование **S** (подстановка по таблице соотвестствия)
3) Линейное преобразование **L**, где происходит сдвиг элементов блока на 1 блок, а утраченный после сдвига блок восполняется путем свертки всех блоков в один в результате линейного преобразования

При зашифровке операции XSL производятся 9 раз (раундов), а 10-й раунд включает только операцию наложения раундового ключа. Расшифрование представляет собой последовательное применение обратных процедур. Подборнее алогитм описывается в Части 3.

### Часть 3. Описание программной реализации

Программа реализована в классе `Cricket` в файле cricket.py в данном репозитории.

* Статические параметры `pi` и `pi_inv`:
    Целочисленные массивы, где индекс каждого элемента соответствует значению исходного при биективном нелинейном отображении (X) и обратной опреации (X<sup>-1</sup>). 

* Метод `__init__(self):`
    Запускается при инициализации класса. Принимает на вход главный ключ (256 bit), запускает генерацию раундовых ключей и сохраняет их в виде упорядоченного массива в параметре объекта `self.round_keys`
    
* Статический метод `__generate_round_keys(key)`: 
    Принимает на вход главный 256 битный ключ и генерирует из него 10 раундовых 128 битных ключей.
    
    а) На первом метод разбивает глвный ключ на две равные части, и сохраняет их в массив `self.round_keys` в качестве первых двух ключей.
    
    б) Остальные 8 (4 пары) ключей вырабатываются в цикле `for i in range(4):`, где на каждом шаге цикла к предыдущей паре ключей 8-кратно применяются преобразования сети Фейстеля. 
    
* Метод `encrypt(self, x)`:
    Принимая на вход блок длинной 128 бит, затем в цикле `for rnd in range(9):` выполняет 9 раундов зашифрования.
    
    а) Запускает сначала операцию **X** (XOR) с райндовым ключом 
    
    `x ^ self.round_keys[rnd]`
    
    б) Затем нелинейное биективное преобразование **S** 
    
    `Cricket.__s_transformation(x ^ self.round_keys[rnd])`
    
    в) И наконец линейное преобразование **L** 
    
    ```Cricket.__l_transformation(Cricket.__s_transformation(x ^ self.round_keys[rnd]))``` 
    
    где каждый байт 16 раз помножается на соответствующий ему элемент поля Галуа. В программе этот шаг реализован в методе `__linear_function(x)` 
    
    г) Последний, 10 раунд зишифрования является не полным и состоит только из наложения последнего раундового ключа 
    
    ` return x ^ self.round_keys[-1]`

    
* Расшифрование `def decrypt(self, x)` устроено противоположным образом. Метод принимает на вход зашифрованный блок длинной 128 бит. 

    а) Сначала строится развернутый массив ключей 
    
    `keys = self.round_keys[::-1]`
    
    б) Затем 9 раз применяются обратные функции **X<sup>-1</sup>** **S<sup>-1</sup>** **L<sup>-1</sup>** 
    
    `x = Cricket.__s_inv_transformation(Cricket.__l_inv_transformation(x ^ keys[rnd]))`
    
    в) Последний раунд также является не полным и заключается лишьт в применении последнего раундового ключа
    
    `return x ^ keys[-1]`

### Часть 4. Демонстрация работы программы

1) Чтобы запустить код выполните следующие действия:


```evgeny@hp:~/cricket$ python3 cricket.py <command> <mode> <path/to/file> <key>``` , где:


<br>

`<command>` - <i>encrypt</i> - чтобы зашифровать файл, <i>decrypt</i> - чтобы расшифровать

`<mode>` - режим шифрования / расшифрования

`<path/to/file>` - путь к файлу. При зашифровании имя зашифрованного файла будет содержать дополнительное расширение `.enc`. При расшифровании - данное расширение, если оно имеется, будет удалено 

`<key>` - 256-битный ключ в виде строки

In [1]:
# Выведем на консоль все файлы текущей директории
!ls -l

total 24
-rw-rw-r-- 1 evgeny evgeny  6813 Mar 23 14:42 cricket.py
-rw-rw-r-- 1 evgeny evgeny 11122 Mar 23 16:29 report.ipynb
-rw-rw-r-- 1 evgeny evgeny   453 Mar 23 16:29 test.txt


In [2]:
# Запускаем зашифрование файла test.txt
!python3 cricket.py

In [3]:
!ls -l

total 32
-rw-rw-r-- 1 evgeny evgeny  6813 Mar 23 14:42 cricket.py
-rw-rw-r-- 1 evgeny evgeny   448 Mar 23 16:29 decrypted.txt
-rw-rw-r-- 1 evgeny evgeny   448 Mar 23 16:29 encrypted.txt
-rw-rw-r-- 1 evgeny evgeny 11122 Mar 23 16:29 report.ipynb
-rw-rw-r-- 1 evgeny evgeny   453 Mar 23 16:29 test.txt
